# Pre-processing of YTF
Including 
face recognition  
image cropping  
randomly picking one image from evey vedio files  

In [85]:
import cv2
from skimage import io
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
#%matplotlib inline
import os
import scipy.misc
import random

In [ ]:
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
dataset_path = '/home/danning/face_verif/aligned_images_DB/'

# Average face
An idea of compute average faces of every set of images of every person:  
However it works out really bad   

In [35]:
#This method works out very bad
def crop_and_avg():
    for subdir, dirs, files in os.walk(dataset_path): 
        for names in dirs:
            for subdir2, dirs2, files2 in os.walk(dataset_path+names+'/'):
                for numbers in dirs2:
                    for subdir3, dirs3, files3 in os.walk(dataset_path+names+'/'+numbers+'/'):
                        size = len(files3)
                        avg_img = np.zeros((200,200, 3))
                        for imgs in files3:
                            image = Image.open(dataset_path+names+'/'+numbers+'/'+imgs)
                            gray = cv2.cvtColor(np.asarray(image), cv2.COLOR_BGR2GRAY)
                            faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                            if len(faces)==1:
                                (x, y, w, h) = faces[0]
                                center_x = x+w/2
                                center_y = y+h/2
                                b_dim = min(max(w,h)*1.2,image.width, image.height) # WARNING : this formula in incorrect

                                box = (int(center_x-b_dim/2), int(center_y-b_dim/2), int(center_x+b_dim/2), int(center_y+b_dim/2))
                                # Crop Image
                                crpim = image.crop(box)
                            
                                resim = scipy.misc.imresize(crpim, size=(200,200,3),interp='nearest')
                            
                                if not os.path.exists('/home/danning/face_verif/ytf_cropped_faces/'+names+'/'+numbers):
                                    os.makedirs('/home/danning/face_verif/ytf_cropped_faces/'+names+'/'+numbers)
                                resim = Image.fromarray(resim.astype(np.uint8))     
                                resim.save('/home/danning/face_verif/ytf_cropped_faces/'+names+'/'+numbers+'/'+imgs)
                            
                                #average face
                                avg_img += np.asarray(crpim)
                        avg_img /= size
                        avg_img = Image.fromarray(avg_img.astype(np.uint8))
                        avg_img.save('/home/danning/face_verif/ytf_avg_faces/'+names+'_'+numbers+'.jpg')
                        print('Average face of {0} {1} saved'.format(names,numbers))
        print('Saved images for {0}'.format(names))

# Pick only one image from every set:
An idea of randomly pick only one image from every set: randomly pick one , recognize the face and crop:
However the faces of some image sets are hard for opencv to recognize, or it finds more than one faces.  
I put all the 'bad' quality sets' names in a list called 'bad_files'.  
I crop the faces of bad files by screenshot tool, then use a method 'bad_files_resize' to resize to the same size, (200,200,3).  


In [64]:
def random_pick_and_crop():
    bad_files = []
    for subdir, dirs, files in os.walk(dataset_path):
        for names in dirs:
            for subdir2, dirs2, files2 in os.walk(dataset_path+names+'/'):
                for numbers in dirs2:
                    for subdir3, dirs3, files3 in os.walk(dataset_path+names+'/'+numbers+'/'):
                        faces = []
                        count = 0
                        while len(faces)!=1:
                            count +=1
                            if count>10:
                                bad_files.append(names+numbers)
                                break
                            print('stuck at {0} {1}'.format(names,numbers))
                            image = Image.open(dataset_path+names+'/'+numbers+'/'+files3[random.randrange(len(files3))])
                            
                            gray = cv2.cvtColor(np.asarray(image), cv2.COLOR_BGR2GRAY)
                            faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                            
                        if len(faces)==0:
                            bad_files.append(names+numbers)
                            break
                        (x, y, w, h) = faces[0]#or random one 
                        
                        center_x = x+w/2
                        center_y = y+h/2
                        b_dim = min(max(w,h)*1.2,image.width, image.height) # WARNING : this formula in incorrect
                        box = (int(center_x-b_dim/2), int(center_y-b_dim/2), int(center_x+b_dim/2), int(center_y+b_dim/2))
                        # Crop Image
                        crpim = image.crop(box)
                        resim = scipy.misc.imresize(crpim, size=(200,200,3),interp='nearest')
                        resim = Image.fromarray(resim.astype(np.uint8))
                        resim.save('/home/danning/face_verif/ytf_one_faces/'+names+'_'+numbers+'.jpg')
                        print('Random face of {0} {1} saved'.format(names,numbers))
            print('Saved images for {0}'.format(names))
    print('Done!')
    print('Bad Files:')
    print(bad_files)
    #bad files:['Kyle_Shewfelt0', 'Kyle_Shewfelt0', 'Eliane_Karp0', 'Prince_Charles0', 'Alison_Lohman2', 'Tom_Koenigs3', 'Marie-Josee_Croze2', 'Rand_Beers5', 'Rand_Beers5', 'Alexander_Downer4', 'Charles_Bronson0', 'Paul_Gascoigne4', 'Richard_Lugar1', 'Eladio_Larez3', 'Poala_Suarez5', 'Poala_Suarez5', 'Beyonce_Knowles3', 'Lenny_Wilkens3']


In [ ]:
def bad_files_resize():
    for subdir, dirs, files in os.walk('/home/danning/face_verif/bad files/'):
        for imgs in files:
            image = Image.open('/home/danning/face_verif/bad files/'+imgs)
            resim = scipy.misc.imresize(image, size=(200,200,3),interp='nearest')
            resim = Image.fromarray(resim.astype(np.uint8))
            resim.save('/home/danning/face_verif/ytf_one_faces/'+imgs)
            print('{0} saved '.format(imgs))